# Face tracking pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and tracking on an image dataset using MTCNN.

In [ ]:
# !pip3 install --upgrade pip 
#!pip3 install mmcv
# !pip3 install ffmpeg-python
#!pip3 install facenet_pytorch
!pip3 install MTCNN

distutils: /usr/local/lib/python3.6/dist-packages
sysconfig: /usr/lib/python3.6/site-packages
distutils: /usr/local/lib/python3.6/dist-packages
sysconfig: /usr/lib/python3.6/site-packages
distutils: /usr/local/include/python3.6/UNKNOWN
sysconfig: /usr/include/python3.6m/UNKNOWN
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |################################| 2.3 MB 2.6 MB/s eta 0:00:01
     |################################| 34.2 MB 555 kB/s  eta 0:00:01
     |################################| 371 kB 23.2 MB/s eta 0:00:01
  distutils: /usr/lib/python3/dist-packages
  sysconfig: /usr/lib/python3.6/site-packages
  user = False
  home = None
  root = None
  prefix = None
  distutils: /usr/lib/python3/dist-packages
  sysconfig: /usr/lib/python3.6/site-packages
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... error
  ERROR:

In [6]:
import mmcv, cv2

#from models import mtcnn as MTCNN
import MTCNN
import torch
import numpy as np

from PIL import Image, ImageDraw
from IPython import display

#### Determine if an nvidia GPU is available

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [7]:
mtcnn = MTCNN(keep_all=True, device=device)

TypeError: 'module' object is not callable

#### Get a sample video

We begin by loading a video with some faces in it. The `mmcv` PyPI package by mmlabs is used to read the video frames (it can be installed with `pip install mmcv`). Frames are then converted to PIL images.

In [ ]:
video = mmcv.VideoReader('trimmed.mp4')
frames = list()
# frames = [(idx, Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))) 
#           for frame in enumerate(video) if idx % 100 == 0]


for idx, frame in enumerate(video):
    if idx % 30 == 0:
        frames.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

print(len(frames))

# display.Video('trimmed.mp4', width=640)

#### Run video through MTCNN

We iterate through each frame, detect faces, and draw their bounding boxes on the video frames.

In [ ]:
frames_tracked = []
for idx_frame, frame in enumerate(frames):
    print(f'Tracking frame: {idx_frame}')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    for idx_box, box in enumerate(boxes):
        cropped_img = frame.crop(box)
        
        if idx_box == 3:
            display.display(cropped_img, display_id=True)
            cropped_img.save(f'./{idx_box}/{idx_frame}.jpg')
        
    
#     # Draw faces
#     frame_draw = frame.copy()
#     draw = ImageDraw.Draw(frame_draw)
#     for box in boxes:
#         draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
#     # Add to frame list
#     frames_tracked.append(frame_draw.resize((1280, 720), Image.BILINEAR))
print('\nDone')

#### Display detections

In [ ]:
d = display.display(frames_tracked[0], display_id=True)
i = 1
try:
    while True:
        d.update(frames_tracked[i % len(frames_tracked)])
        i += 1
except KeyboardInterrupt:
    pass

#### Save tracked video

In [ ]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()